# Analysis of type 2 diabetes (T2D)

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in the T2D trait, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`


# 07/01/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

## Bash variables for workflow configuration

In [1]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/T2D
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/T2D
lmm_sos=../workflow/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_T2D-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_T2D-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_T2D-saige.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720
## LMM variables 
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720
LDscoresFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genetic_map_hg19_withX.txt.gz
phenoCol=T2D
covarCol=SEX
covarMaxLevels=10
qCovarCol=AGE
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/T2D
clumping_sos=../workflow/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_T2D_ldclumping.sbatch
## LD clumping variables
# For sumtastsFiles if more than one provide each path
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# Changes dependending upon which traits are analyzed
# In this case T2D and INT-WHR
sumstatsFiles="/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz \
              /gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/ukb_imp_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020.BoltLMM.snp_stats.all_chr.gz"
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/T2D
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_T2D-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/asthma_INT-WHR_T2D/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
extract_job_size=10

## BoltLMM job

In [ ]:
lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

## FastGWA job

In [ ]:
lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"


## SAIGE job

In [ ]:
lmm_args="""SAIGE
    --cwd $lmm_dir_saige 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_saige 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --bgenMinMAC $bgenMinMAC
    --trait_type $trait_type
    --loco $loco
    --sampleCol $sampleCol
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_saige \
    --args "$lmm_args"

## LD clumping job

In [ ]:
clumping_args="""default 
    --cwd $clumping_dir 
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## Region extraction job

In [4]:
extract_args="""default
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile_fastgwa
    --unrelated-samples $unrelated_samples
    --job-size $extract_job_size
"""

sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb dewan \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

INFO: Running dewan: Configuration for Yale `pi_dewan` partition cluster
INFO: dewan is completed.
INFO: dewan output:   ../output/2021-05-03_T2D-region.sbatch
INFO: Workflow dewan (ID=wf6e25133daf5176d) is executed successfully with 1 completed step.



## Hudson Plots

### T2D vs INT-BMI

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-BMI
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-BMI_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-BMI/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720_rankNorm_BMI.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-BMI_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-BMI'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

### T2D vs INT-WHR

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-WHR
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-WHR_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020_rankNorm_WHR.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-WHR_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-WHR'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

### T2D vs INT-WAIST

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-WAIST
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-WAIST_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WAIST/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WAIST_withagesex_042020_rankNorm_WAIST.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-WAIST_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-WAIST'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

# 08/22/22 - Burden test for T2D

## White European extended

In [4]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_extwhite
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_extwhite/T2D_200k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_white_euro
covarFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_white_euro
phenoCol=T2D
covarCol=SEX
qCovarCol='AGE PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10'
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/010622_exp_white_finalqc/cache/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=20
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_extwhite/T2D_200k_exomes-regenie-burden_2022-08-22.sbatch
INFO: Workflow csg (ID=w457b9b9624f4983d) is executed successfully with 1 completed step.


## Asian

In [5]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_asian
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_asian/T2D_200k_exomes-regenie-burden_asian$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_asian
covarFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_asian
phenoCol=T2D
covarCol=SEX
qCovarCol='AGE PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10' 
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/021722_asian_finalqc_10157samples_444076variants/cache/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_asian/T2D_200k_exomes-regenie-burden_asian2022-08-22.sbatch
INFO: Workflow csg (ID=w066ac1a56b6201ed) is executed successfully with 1 completed step.


## African

In [6]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_african
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/082222_T2D_african/T2D_200k_exomes-regenie-burden_african_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_african
covarFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_african
phenoCol=T2D
covarCol=SEX
qCovarCol='AGE PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10'
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/021722_african_finalqc_8591samples_351690variants/cache/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/08222_T2D_african/T2D_200k_exomes-regenie-burden_african_2022-08-22.sbatch
INFO: Workflow csg (ID=wc4ef86ff0f6ea370) is executed successfully with 1 completed step.


# 08/22/22 - Exome analysis T2D Single variant analysis 

## White European extended

In [14]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_extwhite/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_extwhite/T2D_200k_exomes-regenie_extwhite_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_white_euro
covarFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_white_euro
phenoCol=T2D
covarCol=SEX
qCovarCol='AGE PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10' 
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/010622_exp_white_finalqc/cache/*.bed
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_extwhite/T2D_200k_exomes-regenie_extwhite_2022-08-22.sbatch
INFO: Workflow csg (ID=w13150adf623ac7ba) is executed successfully with 1 completed step.


## Asian

In [15]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_asian/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_asian/T2D_200k_exomes-regenie_asian_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_asian
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE PC{1..10}`
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/021722_asian_finalqc_10157samples_444076variants/cache/UKB_genotypedatadownloaded083019.021722_asian_finalqc_10157samples_444076variants.filtered.extracted.bed
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions
## After doing the annotation you can create the annotatted manhatan plot
#anno_file=$lmm_exome_dir_regenie/090921_f3393_hearing_aid_200K/091321_annotation/*.formatted.csv
#label_annotate=Gene
#known_vars=
#new_vars=
# If --annotate then it will add the label to the plot otherwise --no-annotate

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_asian/T2D_200k_exomes-regenie_asian_2022-08-22.sbatch
INFO: Workflow csg (ID=w5fb01359ff79f6f9) is executed successfully with 1 completed step.


## African 

In [16]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_african/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_african/T2D_200k_exomes-regenie_african_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/pheno_covs_t2d_african
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE PC{1..10}`
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/021722_african_finalqc_8591samples_351690variants/cache/*.bed
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions

## After doing the annotation you can create the annotatted manhatan plot
#anno_file=$lmm_exome_dir_regenie/090921_f3393_hearing_aid_200K/091321_annotation/*.formatted.csv
#label_annotate=Gene
#known_vars=
#new_vars=
# If --annotate then it will add the label to the plot otherwise --no-annotate

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/082222_T2D_african/T2D_200k_exomes-regenie_african_2022-08-22.sbatch
INFO: Workflow csg (ID=w7bbf6cdb501e6971) is executed successfully with 1 completed step.


# 04/12/23 Burden Test for T2D (using hg38 genotype array file)

## White European

In [1]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_whiteEUR-hg38
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_whiteEUR-hg38/T2D_EUR_200kWES-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_white_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_white_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/012323_white_european_460649ind_hg38/final_files_no_outliers/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg2.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg2 \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg2: Configuration for Columbia csg partition cluster
INFO: csg2 is completed.
INFO: csg2 output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_whiteEUR-hg38/T2D_EUR_200kWES-regenie-burden_2023-04-12.sbatch
INFO: Workflow csg2 (ID=w0ced246e9a398263) is executed successfully with 1 completed step.


## Asian

In [2]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_asian-hg38
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_asian-hg38/T2D_ASN_200kWES-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_asian_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_asian_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/012323_asian_10189ind_hg38/final_files_no_outliers/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg2.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg2 \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg2: Configuration for Columbia csg partition cluster
INFO: csg2 is completed.
INFO: csg2 output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_asian-hg38/T2D_ASN_200kWES-regenie-burden_2023-04-12.sbatch
INFO: Workflow csg2 (ID=wb08e7801c8f49c8f) is executed successfully with 1 completed step.


## African

In [1]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_african-hg38
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_african-hg38/T2D_AFR_200kWES-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_african_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_african_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/012323_african_9096ind_hg38/final_files_no_outliers/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg2.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg2 \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg2: Configuration for Columbia csg partition cluster
INFO: csg2 is completed.
INFO: csg2 output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/041223_T2D_african-hg38/T2D_AFR_200kWES-regenie-burden_2023-04-14.sbatch
INFO: Workflow csg2 (ID=w4bd241896fec852f) is executed successfully with 1 completed step.


# 04/12/23 - Exome analysis T2D Single variant analysis (using hg38 genotype array file)

## White European

In [7]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_whiteEUR/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_whiteEUR/T2D_whiteEUR_200kWES-regenie_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_white_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_white_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/012323_white_european_460649ind_hg38/final_files_no_outliers/*.bed
tpl_file=~/project/bioworkflows/admin/csg2.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg2\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg2: Configuration for Columbia csg partition cluster
INFO: csg2 is completed.
INFO: csg2 output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_whiteEUR/T2D_whiteEUR_200kWES-regenie_2023-04-12.sbatch
INFO: Workflow csg2 (ID=wa5e41c0ed85e8631) is executed successfully with 1 completed step.


## Asian

In [8]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_ASN/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_ASN/T2D_ASN_200kWES-regenie_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_asian_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_asian_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/012323_asian_10189ind_hg38/final_files_no_outliers/*.bed
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_ASN/T2D_ASN_200kWES-regenie_2023-04-12.sbatch
INFO: Workflow csg (ID=wa76cc4d031e7da70) is executed successfully with 1 completed step.


## African

In [2]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_AFR/
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_AFR/T2D_AFR_200kWES-regenie$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_african_T2D_pcs_rerun
covarFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_african_T2D_pcs_rerun
phenoCol=T2D
covarCol=SEX
qCovarCol=`echo AGE T2D_PC{1..10}`
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=~/UKBiobank/genotype_files_processed/012323_african_9096ind_hg38/final_files_no_outliers/*.bed
tpl_file=~/project/bioworkflows/admin/csg2.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
## Using this MAC the default in regenie analysis
minMAC=5
label_annotate='SNP'
lowmem_dir=~/scratch60/predictions

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --label_annotate $label_annotate
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg2\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg2: Configuration for Columbia csg partition cluster
INFO: csg2 is completed.
INFO: csg2 output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/041223_T2D_AFR/T2D_AFR_200kWES-regenie2023-04-12.sbatch
INFO: Workflow csg2 (ID=wa6b5e24abfaf347c) is executed successfully with 1 completed step.
